In [1]:
import requests
import datetime

In [2]:
ENV = 'stg'

In [3]:
PROMPT_CONTROL_MAP = {
    'stg' : "172.18.243.211",
    'prd' : "172.27.10.166",
}

In [4]:
PROMPT_ID_MAP = {
    'stg' : '123954',
    'prd' : None,
}

In [46]:
## anaylze_requested_movie_meta function 프롬프트
get_movie_information_system_template = """You are a smart assistant for 에이닷 application user. 
Your main goals are:
- You must use 'keyword_analysis' function to analyze user intent from history of conversations with user.
- You must extract required arguments for function calling from history of conversations with user.
- Required arguments must included.
- When extracting arguments, you must only refer to the user's contents. Never make assumptions.
- If you are unable to extract some arguments, you should use function without them.
- Never engage in a general conversation with the user. Use function calling"""

functions = [
    {
        'name': 'keyword_analysis',
        'description': """Useful to analyze keywords related to the movie from history of conversations with user.""",
        "parameters": {
            "type": "object",
            "properties": {
                "has_specific_keyword": {
                    "type": "string",
                    "enum": ["yes", "no"],
                    "description": "Whether the user's question contains specific search keywords for movies such as actor, genre, content, characteristic, etc.. Common keywords such as '재미있는', '유명한', etc. are not specific keywords."
                },
                "actor": {
                    "type": "array",
                    "description": "The actor who appeared in the movie",
                    "items": {
                        "type": "string",
                    },
                },
                "title": {
                    "type": "string",
                    "description": "The title of movie",
                },
                "genre": {
                    "type": "string",
                    "description": "The genre of movie",
                    "items": {
                        "type": "string",
                    },
                },
                "nation": {
                    "type": "string",
                    "description": "The country of movie",
                },
                "content": {
                    "type": "string",
                    "description": "The content of movie",
                },
                "order": {
                    "type": "string",
                    "description": "Sorting order by movie rating",
                    "enum": ['descending', 'ascending'],
                },
            },
            'required': ['has_specific_keyword'],
        },
    }
]
def anaylze_requested_movie_meta(question:str):
    
    messages = [
        {"role": "system", "content": get_movie_information_system_template},
        {"role": "user", "content": question},
    ]
    
    headers = {
        "Accept": "application/json",
    }
    
    history = messages[:-1]
    message = messages[-1]

    data = {
        "promptId": PROMPT_ID_MAP[ENV],
        "requestTexts": [
            {
                "key": "content",
                "value": message['content'],
            },
        ],
        'role': message['role'],
        'history' : history,
        "functionCall" : {
            "control": "auto",
            "functions" : functions,
            "name" : "keyword_analysis",
        }
    }

    resp = requests.post(
        f"http://{PROMPT_CONTROL_MAP[ENV]}:13100/api/v1/completions",
        headers=headers,
        json=data,
    )
    resp = resp.json()
    
    return resp

In [47]:
text = "타이타닉 보고 싶다."
anaylze_requested_movie_meta(text)

{'state': 200,
 'res': {'transactionId': '202311061953af712360424395b81e2db076cf6bce',
  'model': 'gpt-3.5-turbo',
  'usage': {'promptTokens': 303, 'completionTokens': 21, 'totalTokens': 324},
  'functionCall': {'name': 'keyword_analysis',
   'arguments': '{\n  "title": "타이타닉"\n}'}}}

In [48]:
text = "나는 요즘 공포 영화가 보고 싶은데 추천해줘."
anaylze_requested_movie_meta(text)

{'state': 200,
 'res': {'transactionId': '2023110619ff5a78801d3448a5bbc7742fa850b5fa',
  'model': 'gpt-3.5-turbo',
  'usage': {'promptTokens': 318, 'completionTokens': 18, 'totalTokens': 336},
  'functionCall': {'name': 'keyword_analysis',
   'arguments': '{\n  "genre": "공포"\n}'}}}

In [49]:
text = "재미있는 놀이는 뭘까?"
anaylze_requested_movie_meta(text)

{'state': 200,
 'res': {'transactionId': '202311061925ecea1cf4b04c0e8b4c8a00385c0f02',
  'model': 'gpt-3.5-turbo',
  'usage': {'promptTokens': 306, 'completionTokens': 17, 'totalTokens': 323},
  'functionCall': {'name': 'keyword_analysis',
   'arguments': '{\n  "has_specific_keyword": "yes"\n}'}}}

In [50]:
text = "잠과 관련된 영화"
anaylze_requested_movie_meta(text)

{'state': 200,
 'res': {'transactionId': '2023110619db5e219eebbb4fe4ae1a97bc8facad9f',
  'model': 'gpt-3.5-turbo',
  'usage': {'promptTokens': 300, 'completionTokens': 25, 'totalTokens': 325},
  'functionCall': {'name': 'keyword_analysis',
   'arguments': '{\n  "has_specific_keyword": "yes",\n  "content": "잠"\n}'}}}